In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, Column, join, MaskedColumn
from astropy import units as u

In [2]:
table = Table.read('gr8.csv')

In [3]:
for inst in ['harps', 'hermes', 'sophie']: # 'feros', 'fies', 'uves']:
    table.remove_columns([f'spectrum_{inst}_1d', f'spectrum_{inst}_snr', f'spectrum_{inst}_nobs'])
    try:
        table.remove_column(f'spectrum_{inst}_2d')
    except:
        continue

In [4]:
# strip the whitespace:
table['primary_name'] = [t.replace(" ","") for t in table['primary_name']]

In [5]:
def update_inst(inst, inst_table, table, debug_mode=False):
    # I'm reading the table outside this loop bc there are formatting inconsistencies
    tmp_table = inst_table[:]
    tmp_table.rename_column('Name', 'primary_name')
    tmp_table['primary_name'] = [t.replace(" ","") for t in tmp_table['primary_name']] # whitespace
    tmp_table.rename_column('S2D {0}'.format(inst.upper()), f'spectrum_{inst}_nobs')
    tmp_table.rename_column('SNR co-added {0}'.format(inst.upper()), f'spectrum_{inst}_snr')
    # update the main table:
    main_table = table[:]
    main_table.add_column(Column(name=f'spectrum_{inst}', dtype='<U29', length=len(main_table)))
    main_table = join(main_table, tmp_table, keys='primary_name', join_type='left')
    main_table[f'spectrum_{inst}_nobs'].fill_value = 0
    main_table[f'spectrum_{inst}_snr'].fill_value = 0.
    main_table[f'spectrum_{inst}'][(main_table[f'spectrum_{inst}_snr'] > 0) 
                               & ~(main_table[f'spectrum_{inst}_snr'].mask)] = 'true'
    # check we got them all:
    if debug_mode:
        return join(main_table, tmp_table, keys='primary_name', join_type='right')
    assert len(main_table[main_table[f'spectrum_{inst}'] == 'true']) == len(tmp_table)
    return main_table 

### update FEROS:

NOTE: I had to change "phi.02 cet" to "phi.02 Cet". Also one of the SNRs had 'SS' at the end which I deleted.

In [6]:
feros_table = Table.read('alix/Gr8_spectra_info_FEROS.txt', format='ascii')

In [7]:
try:
    table = update_inst('feros', feros_table, table)
except:
    print('Failure!!')
    debug_table = update_inst('feros', feros_table, table, debug_mode=True)

In [8]:
#debug_table[debug_table['spectrum_feros'] != 'true']

### update FIES:

NOTE: I had to change Gaia EDR3 892215482207937152 to HIP36357.

In [9]:
fies_table = Table.read('alix/Gr8_spectra_info_FIES.txt', format='ascii', delimiter='\t')
fies_table['S2D FIES'] = 1

In [10]:
try:
    table = update_inst('fies', fies_table, table)
except:
    print('Failure!!')
    debug_table = update_inst('fies', fies_table, table, debug_mode=True)

In [11]:
#debug_table[debug_table['spectrum_fies'] != 'true']

### update HARPS:

In [12]:
harps_table = Table.read('alix/Gr8_spectra_info_HARPS.txt', format='ascii', 
                         delimiter='\t', comment='-')

In [13]:
assert len(np.unique(harps_table['Name'])) == len(harps_table)

In [14]:
names = harps_table['Name']
u, c = np.unique(names, return_counts=True)
print("Repeat entries:")
print(u[c > 1])

Repeat entries:
Name
----


In [15]:
try:
    table = update_inst('harps', harps_table, table)
except:
    print('Failure!!')
    debug_table = update_inst('harps', harps_table, table, debug_mode=True)

In [16]:
#debug_table[debug_table['spectrum_harps'] != 'true']

### update HERMES:

Had to change pi01 UMa to pi.01 UMa.

In [17]:
hermes_table = Table.read('alix/Gr8_spectra_info_HERMES.txt', format='ascii', 
                          delimiter='\t', comment='-')['Name', 'SNR co-added HERMES']
hermes_table['S2D HERMES'] = 1

In [18]:
assert len(np.unique(hermes_table['Name'])) == len(hermes_table)

In [19]:
try:
    table = update_inst('hermes', hermes_table, table)
except:
    print('Failure!!')
    debug_table = update_inst('hermes', hermes_table, table, debug_mode=True)

In [20]:
#print(debug_table[debug_table['spectrum_hermes'] != 'true']['primary_name'])

### update SOPHIE:

In [21]:
sophie_table = Table.read('alix/Gr8_spectra_info_SOPHIE.txt', format='ascii', comment='-')

In [22]:
assert len(np.unique(sophie_table['Name'])) == len(sophie_table)

In [23]:
names = sophie_table['Name']
u, c = np.unique(names, return_counts=True)
print("Repeat entries:")
print(u[c > 1])

Repeat entries:
Name
----


In [24]:
try:
    table = update_inst('sophie', sophie_table, table)
except:
    print('Failure!!')
    debug_table = update_inst('sophie', sophie_table, table, debug_mode=True)

In [25]:
#debug_table[debug_table['spectrum_sophie'] != 'true']

### update UVES:

In [26]:
uves_table = Table.read('alix/Gr8_spectra_info_UVES.txt', format='ascii', delimiter='\t')#['Name', 'SNR co-added UVES']
#uves_table['S2D UVES'] = 1 # hack??

In [27]:
try:
    table = update_inst('uves', uves_table, table)
except:
    print('Failure!!')
    debug_table = update_inst('uves', uves_table, table, debug_mode=True)

In [28]:
#debug_table[debug_table['spectrum_uves'] != 'true']

### save it:

In [29]:
table.write('gr8_updated.csv', overwrite=True)